In [ ]:
# Complete Case kNN
start = time.time()
accuracy_cc_kNN = 0
for i in range(N_test):
    if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_train,Y_train,O_train):
        accuracy_cc_kNN += 1/N_test
end = time.time()
run_time_cc_kNN = end-start
print('Runtime Complete Case kNN (in s): ',np.round(run_time_cc_kNN,3))
print('    Accuracy: ',np.round(accuracy_cc_kNN,3))

if not (O_train==np.ones((N_train,d))).all():
    # Mean Imputation then kNN
    start = time.time()
    X_imputed = mean_impute(X_train, O_train)
    accuracy_mi_kNN = 0
    for i in range(N_test):
        if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_imputed,Y_train,np.ones((N_train,d))):
            accuracy_mi_kNN += 1/N_test
    end = time.time()
    run_time_mi_kNN = end-start
    print('Runtime Mean Imputation and kNN (in s): ',np.round(run_time_mi_kNN,3))
    print('    Accuracy: ',np.round(accuracy_mi_kNN,3))

    # c=0 Imputation then kNN
    start = time.time()
    X_imputed = constant_impute(X_train, O_train)
    accuracy_ci_kNN = 0
    for i in range(N_test):
        if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_imputed,Y_train,np.ones((N_train,d))):
            accuracy_ci_kNN += 1/N_test
    end = time.time()
    run_time_ci_kNN = end-start
    print('Runtime Constant Imputation and kNN (in s): ',np.round(run_time_ci_kNN,3))
    print('    Accuracy: ',np.round(accuracy_ci_kNN,3))
    

    # MissForest imputation using missingpy
    start = time.time()
    imputer = MissForest(criterion=('squared_error', 'gini'),max_features=1.0)
    X_miss = np.zeros((N_train,d))
    X_miss += X_train
    for i in range(N_train):
        for j in range(d):
            if O_train[i,j]==0:
                X_miss[i,j] = 'nan'
    sys.stdout = open(os.devnull, 'w')
    X_imputed = imputer.fit_transform(X_miss)
    sys.stdout = sys.__stdout__
    accuracy_mf_kNN = 0
    for i in range(N_test):
        if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_imputed,Y_train,np.ones((N_train,d))):
            accuracy_mf_kNN += 1/N_test
    end = time.time()
    run_time_mf_kNN = end-start
    print('Runtime MissForest and kNN (in s): ',np.round(run_time_mf_kNN,3))
    print('    Accuracy: ',np.round(accuracy_mf_kNN,3))


# Oracle HAM
start = time.time()
accuracy_oracle_HAM = 0
for i in range(N_test):
    if y_test[i] == HAM_classifier(x_test[i,:], Omega_star, X_train, Y_train, O_train, alpha, betas, gammas):
        accuracy_oracle_HAM += 1/N_test
end = time.time()
run_time_oracle_HAM = end-start
print('Runtime Oracle HAM (in s): ',np.round(run_time_oracle_HAM,3))
print('    Accuracy: ',np.round(accuracy_oracle_HAM,3))

# HAM
start = time.time()
Omega_hat = []
Omega_hat = estimate_Omega_star(X_train, Y_train, O_train, alpha, betas, gammas)
accuracy_HAM = 0
for i in range(N_test):
    if y_test[i] == HAM_classifier(x_test[i,:], Omega_hat, X_train, Y_train, O_train, alpha, betas, gammas):
        accuracy_HAM += 1/N_test
end = time.time()
run_time_HAM = end-start
print('Runtime HAM (in s): ',np.round(run_time_HAM,3))
print('    Accuracy: ',np.round(accuracy_HAM,3))

# cvHAM
start = time.time()
Omega_hat = []
r_optimal = estimate_params_CV(X_train, Y_train, O_train)
print('alpha = ',r_optimal[0])
print('betas = ',r_optimal[1])
print('gammas = ',r_optimal[2])
Omega_hat = estimate_Omega_star(X_train, Y_train, O_train, r_optimal[0],r_optimal[1],r_optimal[2])
accuracy_cvHAM = 0
for i in range(N_test):
    if y_test[i] == HAM_classifier(x_test[i,:], Omega_hat, X_train, Y_train, O_train, r_optimal[0],r_optimal[1],r_optimal[2]):
        accuracy_cvHAM += 1/N_test
end = time.time()
run_time_cvHAM = end-start
print('Runtime cvHAM (in s): ',np.round(run_time_cvHAM,3))
print('    Accuracy: ',np.round(accuracy_cvHAM,3))

if not (O_train==np.ones((N_train,d))).all():
    # Fully observed kNN
    start = time.time()
    accuracy_full_kNN = 0
    for i in range(N_test):
        if y_test[i] == complete_case_kNN_classifier(x_test[i,:],X_train,Y_train,np.ones((N_train,d))):
            accuracy_full_kNN += 1/N_test
    end = time.time()
    run_time_full_kNN = end-start
    print('Runtime Full kNN, everything observed (in s): ',np.round(run_time_full_kNN,3))
    print('    Accuracy: ',np.round(accuracy_full_kNN,3))

    # Random Forest Imputation + kNN - Fit on and transform our training data; Only use 2 iterations of mice.
    start = time.time()
    X_miss = np.zeros((N_train,d))
    X_miss += X_train
    for i in range(N_train):
        for j in range(d):
            if O_train[i,j]==0:
                X_miss[i,j] = 'nan'
    # Run the MICE algorithm for 2 iterations and return 5 completed datasets
    kds = mf.ImputationKernel(X_miss, datasets=5)
    kds.mice(2)
    accuracy_mice_kNN = 0
    for i in range(N_test):
        y_s=0
        for s in range(5):
            X_train_t = kds.complete_data(dataset=s)
            y_s += complete_case_kNN_classifier(x_test[i,:],X_train_t,Y_train,np.ones((N_train,d)))/5
        if y_s>=1/2:
            y_s = 1
        else:
            y_s = 0
        if y_test[i] == y_s:
            accuracy_mice_kNN += 1/N_test
    end = time.time()
    run_time_mice_kNN = end-start
    print('Runtime mice and kNN (in s): ',np.round(run_time_mice_kNN,3))
    print('    Accuracy: ',np.round(accuracy_mice_kNN,3))


if (O_train==np.ones((N_train,d))).all():
    print('\nEstimated runtime (in min): ',np.round(iterations*(run_time_cc_kNN+run_time_HAM+run_time_cvHAM+run_time_oracle_HAM)/60,3))
else:
    print('\nEstimated runtime (in min): ',np.round(iterations*(run_time_mice_kNN+run_time_full_kNN+run_time_ci_kNN+run_time_mf_kNN+run_time_cc_kNN+run_time_mi_kNN+run_time_HAM+run_time_cvHAM+run_time_oracle_HAM)/60,3))